## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cherskiy,RU,68.7500,161.3000,0.03,91,99,8.03,overcast clouds
1,1,Saldanha,ZA,-33.0117,17.9442,61.65,83,19,10.83,few clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,46.06,81,40,6.91,scattered clouds
3,3,Dikson,RU,73.5069,80.5464,26.94,83,100,16.06,overcast clouds
4,4,Barmer,IN,25.7500,71.3833,97.32,4,0,4.92,clear sky


In [60]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is minimum temperature for your destination? "))
max_temp = float(input("What is maximum temperature for your destination? "))

What is minimum temperature for your destination? 48
What is maximum temperature for your destination? 105


In [61]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
vacation_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saldanha,ZA,-33.0117,17.9442,61.65,83,19,10.83,few clouds
4,4,Barmer,IN,25.7500,71.3833,97.32,4,0,4.92,clear sky
6,6,Butaritari,KI,3.0707,172.7902,81.39,74,8,9.48,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,77.27,79,71,12.30,light rain
8,8,Punta Arenas,CL,-53.1500,-70.9167,50.11,71,75,26.46,light rain


In [62]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.count()

City_ID                31
City                   31
Country                31
Lat                    31
Lng                    31
Max Temp               31
Humidity               31
Cloudiness             31
Wind Speed             31
Current Description    31
dtype: int64

In [63]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [64]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saldanha,ZA,61.65,few clouds,-33.0117,17.9442,
4,Barmer,IN,97.32,clear sky,25.7500,71.3833,
6,Butaritari,KI,81.39,clear sky,3.0707,172.7902,
7,Rikitea,PF,77.27,light rain,-23.1203,-134.9692,
8,Punta Arenas,CL,50.11,light rain,-53.1500,-70.9167,
9,Rassvet,RU,56.32,scattered clouds,46.8333,40.7500,
11,Georgetown,MY,87.75,few clouds,5.4112,100.3354,
14,Praia Da Vitoria,PT,59.76,broken clouds,38.7333,-27.0667,
15,Dunedin,NZ,62.80,overcast clouds,-45.8742,170.5036,
16,Dakar,SN,67.08,overcast clouds,14.6937,-17.4441,


In [65]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json' 
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        continue
    
        

In [66]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saldanha,ZA,61.65,few clouds,-33.0117,17.9442,Blue Bay Lodge
4,Barmer,IN,97.32,clear sky,25.7500,71.3833,Jain Dharamshala
6,Butaritari,KI,81.39,clear sky,3.0707,172.7902,Isles Sunset Lodge
7,Rikitea,PF,77.27,light rain,-23.1203,-134.9692,People ThankYou
8,Punta Arenas,CL,50.11,light rain,-53.1500,-70.9167,Hotel Hain
9,Rassvet,RU,56.32,scattered clouds,46.8333,40.7500,
11,Georgetown,MY,87.75,few clouds,5.4112,100.3354,Cititel Penang
14,Praia Da Vitoria,PT,59.76,broken clouds,38.7333,-27.0667,Hotel Branco I
15,Dunedin,NZ,62.80,overcast clouds,-45.8742,170.5036,Scenic Hotel Southern Cross
16,Dakar,SN,67.08,overcast clouds,14.6937,-17.4441,Novotel Dakar


In [67]:
# 7. Drop the rows where there is no Hotel Name.
len(hotel_df[hotel_df['Hotel Name'] == ''])
drop_list = hotel_df[hotel_df['Hotel Name']==''].index
hotel_df.drop(index=drop_list,inplace=True)


In [68]:
hotel_df.count()

City                   29
Country                29
Max Temp               29
Current Description    29
Lat                    29
Lng                    29
Hotel Name             29
dtype: int64

In [72]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv('WeatherPy_vacation.csv', index_label="City_ID")

In [70]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [71]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))